In [1]:
import pandas
from scipy import stats
import numpy
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
data = pandas.read_csv("5.txt")
values = {
    i: [
        0 if ((x == 'b') or (x == 'positive'))
        else 1 if ((x == 'x') or (x == 'negative'))
        else 2 for x in data[column]
    ] for i, column in enumerate(data)
}

In [3]:
pandas.DataFrame(values)

,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,2,2,2,1,2,0
1,1,1,1,1,2,2,2,2,1,0
2,1,1,1,1,2,2,2,0,0,0
3,1,1,1,1,2,2,0,2,0,0
4,1,1,1,1,2,2,0,0,2,0
5,1,1,1,1,2,0,2,2,0,0
6,1,1,1,1,2,0,2,0,2,0
7,1,1,1,1,2,0,0,2,2,0
8,1,1,1,1,0,2,2,2,0,0
9,1,1,1,1,0,2,2,0,2,0


На коэффициенты корреляции даже смотреть не будем. Можно воспользоваться таблицами сопряженности.

In [4]:
# Берем код з 4 задачи. В качестве разбиений берем целые числа.

p_values={}
for column_1 in values:
    for column_2 in values:
        if column_1 != column_2:
            if column_1 == 9 or column_2 == 9:
                n = m = 2
            else:
                n = m = 3
            nu = numpy.array([[
                len([1 for x, y in zip(values[column_1], values[column_2]) if ((x == i) and (y == j))])
                    for j in range(n)
                ]
                for i in range(n)
            ])
            N = numpy.sum(nu)
            ns = numpy.array([sum(nu[i,:]) for i in range(n)])
            ms = numpy.array([sum(nu[:,j]) for j in range(m)])
            for x in ns:
                for y in ms:
                    if x * y / N < 5:
                        print("UPS")
            statistic = N * numpy.sum([[
                    (nu[i][j] - (ns[i] * ms[j]) / N) ** 2 / (ns[i] * ms[j])
                    for j in range(m)
                ]
                for i in range(n)
            ])
            p_values[(column_1, column_2)] = stats.chi2.sf(statistic, df=(n-1)*(m-1))

In [5]:
# Используем маетод Холма множественной проверки гипотез.
answers = multipletests([p_values[x] for x in p_values] , method="holm", alpha=0.05)[0]

In [6]:
answers

array([False, False, False, False,  True, False,  True, False, False,
       False, False,  True, False,  True,  True, False,  True, False,
       False, False,  True, False, False, False,  True, False, False,
       False,  True,  True, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False,  True,  True, False, False,
       False,  True, False, False, False,  True, False, False, False,
        True, False,  True,  True, False,  True, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False], dtype=bool)

In [7]:
print("dependent samples: {}".format([
    (column_1, column_2) for (column_1, column_2), answer in zip(p_values, answers) if answer == True
]))

dependent samples: [(0, 5), (0, 7), (1, 3), (1, 5), (1, 6), (1, 8), (2, 3), (2, 7), (3, 1), (3, 2), (3, 7), (3, 8), (5, 0), (5, 1), (5, 6), (5, 7), (6, 1), (6, 5), (7, 0), (7, 2), (7, 3), (7, 5), (8, 1), (8, 3)]


Видим, что есть зависимые пары выборок. Поэтому гипотезу о независимости в совокупности отвергаем.